In [1]:
!pip install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git


  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/e0/9e/80e2205d138960a49caea391f3710600895dd8292b6868dc9aff7aa593f9/scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 36.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.11.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.11.0 which is incompatible.
libpysal 

In [2]:
# %cd /kaggle/workingview/tortoise-tts
# !pip install transformers==4.29.2
!mkdir /kaggle/working/tortoise-tts/tortoise/voices/muthu

# !pip install -r requirements.txt
!pip install numpy
!pip install tqdm
!pip install rotary_embedding_torch
!pip install transformers==4.29.2
!pip install tokenizers
!pip install inflect
!pip install progressbar
!pip install einops
!pip install unidecode
!pip install scipy
!pip install librosa
!pip install numba
!pip install ffmpeg


  Obtaining dependency information for rotary_embedding_torch from https://files.pythonhosted.org/packages/75/24/4ee455d4a1ea6cb66e937187383c1d28abdf63e4895118a3cbe5c5347517/rotary_embedding_torch-0.3.5-py3-none-any.whl.metadata
  Obtaining dependency information for einops>=0.3 from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einops-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00
  Obtaining dependency information for transformers==4.29.2 from https://files.pythonhosted.org/packages/17/aa/a89864288afe45abe1ab79f002140a20348140e86836d96096d8f8a3bac0/transformers-4.29.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.8 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [3]:
!cp /kaggle/input/tts-dataset/MV/* /kaggle/working/tortoise-tts/tortoise/voices/muthu
!cp /kaggle/input/tts-dataset/mv.txt /kaggle/working/tortoise-tts/tortoise/data

In [4]:
%cd /kaggle/working/tortoise-tts
!python setup.py install

/kaggle/working/tortoise-tts
/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

    

In [5]:
import argparse
import os

import torch
import torchaudio

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, get_voices, load_voices
from tortoise.utils.text import split_and_recombine_text


tts = TextToSpeech(models_dir='.models')

outpath = 'results/longform/'
selected_voices = 'muthu'.split(',')
regenerate = None
if regenerate is not None:
    regenerate = [int(e) for e in regenerate.split(',')]

# Process text
with open("tortoise/data/mv.txt", 'r', encoding='utf-8') as f:
    text = ' '.join([l for l in f.readlines()])
if '|' in text:
    print("Found the '|' character in your text, which I will use as a cue for where to split it up. If this was not"
          "your intent, please remove all '|' characters from the input.")
    texts = text.split('|')
else:
    texts = split_and_recombine_text(text)

for selected_voice in selected_voices:
    voice_outpath = os.path.join(outpath, selected_voice)
    os.makedirs(voice_outpath, exist_ok=True)

    if '&' in selected_voice:
        voice_sel = selected_voice.split('&')
    else:
        voice_sel = [selected_voice]

    voice_samples, conditioning_latents = load_voices(voice_sel)
    all_parts = []
    for j, text in enumerate(texts):
        if regenerate is not None and j not in regenerate:
            all_parts.append(load_audio(os.path.join(voice_outpath, f'{j}.wav'), 24000))
            continue
        gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                                  preset='ultra_fast', clvp_cvvp_slider=.5)
        gen = gen.squeeze(0).cpu()
        torchaudio.save(os.path.join(voice_outpath, f'{j}.wav'), gen, 24000)
        all_parts.append(gen)
    full_audio = torch.cat(all_parts, dim=-1)
    torchaudio.save(os.path.join(voice_outpath, 'combined.wav'), full_audio, 24000)



<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


100% |########################################################################|


Done.


100% |########################################################################|


Done.


...

100%|██████████| 1/1 [03:35<00:00, 215.80s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 1/1 [00:05<00:00,  5.45s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 30/30 [00:04<00:00,  6.50it/s]
